In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import r2_score

import random
from scipy.optimize import minimize, dual_annealing

In [ ]:
nbpath     = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN'
voxel_path = '/home/mgolub4/DLproj/MLTO_2023/0_data/source_MAT_files'

# dbpath = os.path.join(nbpath, 'dyn_data', 'dynamic_static_database_scaled_DEC23.csv')
dbpath = os.path.join(nbpath, 'dyn_data', 'dynamic_static_database_scaled_APR24.csv')


dyndb = pd.read_csv(dbpath)

stress_ser_path = os.path.join(nbpath, 'dyn_data/stress_series_data')

csvs = [p for p in sorted(os.listdir(stress_ser_path)) if p.endswith('untrunc.csv')]


In [ ]:
dyndb_csvs = (dyndb.dyn_file_name_original + '_processed_untrunc').values

In [ ]:
# dyndb['W_pct_error'] = np.abs((dyndb['W_pred']- dyndb['energy_absorbed_g'])/dyndb['energy_absorbed_g'])*100
# dyndb['sig_pl_pct_error'] = np.abs((dyndb['sig_pl_pred'] - dyndb['plateau_stress_g'] )/ dyndb['plateau_stress_g'] )*100

In [ ]:
opt_cols = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt', 'W_pred', 'sig_pl_pred', 'W_pct_error', 'sig_pl_pct_error']

for col in opt_cols:
    dyndb[col] = pd.Series(np.zeros(len(dyndb)))

In [ ]:
# from scipy.optimize import LinearConstraint, Bounds

In [ ]:
# stress_series[(strain >= 0.2) & (strain <= 0.4)]
# stress_series[200:400].mean()

In [ ]:
# A_guess = np.exp(random.randrange(7,12))#10*random.random()
# B_guess = np.exp(random.randrange(16,24))#20*random.random()
# C_guess = random.randrange(1,10)
# m_guess = random.random()
# n_guess = random.randrange(0,20)
# # 
# print(A_guess, B_guess, C_guess, m_guess, n_guess)

In [ ]:
# Define the objective function
def mod_JC(params, strain, stress_series, sig_pl, W, pl_end):
    A, B, C, m, n = params

    stress_series_fit = A * strain**m + B*(strain / (C - strain))**n
    # stress_series_err = np.sum((stress_series - stress_series_fit)**2)
    stress_series_err = np.sum(np.abs(stress_series - stress_series_fit))

    
    sig_pl_pred = np.mean(stress_series_fit[(strain >= 0.2) & (strain <= 0.4)])
    sig_pl_err = np.abs(sig_pl_pred - sig_pl)

    # pl_end_pred = np.where(stress_series >= 1.3*sig_pl)[0][0] #?????
    # pl_end_pred = np.where(np.isclose(stress_series, 1.3*sig_pl, atol=1e8))[0][-1] #?????
    
    pl_end_pred = stress_series_fit[-1:].values[0]   #?????
    pl_end_err = np.abs(stress_series[-1:].values[0]  - pl_end_pred)

    # W_pred = np.trapz(A * strain**m + B * (strain / (1 - strain))**n, strain)
    W_pred = np.trapz(stress_series_fit, strain)

    W_err = np.abs(W_pred - W)

    return sig_pl_err + W_err + pl_end_err + stress_series_err 

# bounds = ([0, None],[0, None], [0,10], [0.1,0.75], [0,20])
bounds = ([0, 1e12],[0, 1e12], [0,10], [0.1,0.75], [0,20])


def optimize_loop(temp = 5230): #method='trust-constr'
    while True: 
        A_guess = np.exp(random.randrange(7,12))
        B_guess = np.exp(random.randrange(16,24))
        C_guess = random.randrange(1,8)
        m_guess = random.uniform(0.1, 0.65)
        n_guess = random.randrange(0,20)

        initial_guess = [A_guess, B_guess, C_guess, m_guess, n_guess]

        result = dual_annealing(mod_JC, bounds=bounds, args=(strain, stress_series, sig_pl, W, pl_end), maxiter=500, initial_temp=temp)

        # result = minimize(mod_JC, initial_guess, args=(strain, stress_series, sig_pl, W, pl_end), bounds=bounds, method=method)

        if np.isnan(result.x).any():
            print("nan found. Retrying")
        else:
            return result
            


In [1]:
temps = []
temp = 5e3
for i in range(10):
    temps.append(temp)    
    # print(temp)
    temp = round(temp*1.1)


In [2]:
temps

[5000.0, 5500, 6050, 6655, 7321, 8053, 8858, 9744, 10718, 11790]

In [ ]:
# for deriving params for fcc_0p35, April 224

# dyndb_csvs = ['fcc_0p35_sr1000_processed_untrunc',]
dyndb_csvs = ['VF_35_64_VF35p92_processed_untrunc',]

In [ ]:
print("\tW_pred\t\tW_actual\t\tpct_error\tsig_pl_pred\t\tsig_pl_actual\t\tpct_error\t\tinstance")


for csv in dyndb_csvs[:1]:
    csvpath = os.path.join(stress_ser_path, f'{csv}.csv')
    df = pd.read_csv(csvpath)

    instance = '_'.join(csv.split('_')[:-2])


    sig_pl = dyndb[dyndb['dyn_file_name_original'] == instance]['plateau_stress_g'].values[0]#/10e6
    W = dyndb[dyndb['dyn_file_name_original'] == instance]['energy_absorbed_g'].values[0]#/10e6

    pl_end = dyndb[dyndb['dyn_file_name_original'] == instance]['end_strain_g'].values[0]
    # pl_end_idx = dyndb[dyndb['Strain'] == pl_end].index

    strn = np.asarray(df['Strain'])
    pl_end_idx = np.where(np.isclose(pl_end, strn))[0][0]

    strain = df['Strain'][:pl_end_idx]
    stress_series = df['stress_bottom_gsreg'][:pl_end_idx]#/10e6

    for i, temp in enumerate(temps):
        result = optimize_loop(temp=temp) #method=method


        A_opt, B_opt, C_opt, m_opt, n_opt = result.x

        
        offset = -0.01
        strs_pred = A_opt * (strain + offset)**m_opt + B_opt*((strain + offset)/(C_opt-(strain + offset)))**n_opt

        W_pred = np.trapz(strs_pred[10:], strn[10:strs_pred.shape[0]])
        sig_pl_pred = np.mean(strs_pred[200:400])

        sig_pl_pct_error = np.abs((sig_pl_pred - sig_pl)/sig_pl)*100
        W_pct_error = np.abs((W_pred - W)/sig_pl)*100


        if sig_pl_pct_error <= 15 and W_pct_error <=15:
            
            opt_params = [A_opt, B_opt, C_opt, m_opt, n_opt, W_pred, sig_pl_pred, W_pct_error, sig_pl_pct_error]
            for opt_col, par in zip(opt_cols, opt_params):
                idx = dyndb[dyndb['dyn_file_name_original'] == instance].index
                dyndb.loc[idx, opt_col] = par

            print(f"optimal results:\ttemp:\t{temp}")
            print(f"\t{W_pred:.3e}\t{W:.3e}\t\t{W_pct_error:.2f}%\t\t{sig_pl_pred:.3e}\t\t{sig_pl:.3e}\t\t{sig_pl_pct_error:.2f}%\t\t{instance}")
            print(f"Optimized parameters: {A_opt:.3e}\t{B_opt:.3e}\t{C_opt:.2f}\t{m_opt:.2f}\t{n_opt:.2f}")

            fig = make_subplots(rows=1, cols=1)

            fig.append_trace(go.Scatter(x = strain, y = stress_series, name='original_data'), row=1, col=1)
            fig.append_trace(go.Scatter(x = strain, y = strs_pred, name='predicted'), row=1, col=1)
            fig.update_layout(width=1000)
            plotpath = os.path.join('/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/fitted_curves_17APR', instance)
            fig.write_image(f'{plotpath}.png')

            break

        elif i == len(temps)-1:

            opt_params = [A_opt, B_opt, C_opt, m_opt, n_opt, W_pred, sig_pl_pred, W_pct_error, sig_pl_pct_error]
            for opt_col, par in zip(opt_cols, opt_params):
                idx = dyndb[dyndb['dyn_file_name_original'] == instance].index
                dyndb.loc[idx, opt_col] = par
            
            print(f"results for last iteration:\ttemp:\t{temp}\t\t)#... as good as it's going to get")
            print(f"\t{W_pred:.3e}\t{W:.3e}\t\t{W_pct_error:.2f}%\t\t{sig_pl_pred:.3e}\t\t{sig_pl:.3e}\t\t{sig_pl_pct_error:.2f}%\t\t{instance}")
            print(f"Optimized parameters: {A_opt:.3e}\t{B_opt:.3e}\t{C_opt:.2f}\t{m_opt:.2f}\t{n_opt:.2f}")
            

            fig = make_subplots(rows=1, cols=1)

            fig.append_trace(go.Scatter(x = strain, y = stress_series, name='original_data'), row=1, col=1)
            fig.append_trace(go.Scatter(x = strain, y = strs_pred, name='predicted'), row=1, col=1)
            fig.update_layout(width=1000)
            plotpath = os.path.join('/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/fitted_curves_17APR', instance)
            fig.write_image(f'{plotpath}.png')



In [ ]:
# print("\tW_pred\t\tW_actual\t\tpct_error\tsig_pl_pred\t\tsig_pl_actual\t\tpct_error\t\tinstance")

# # methods = ['Powell',] #'Nelder-Mead',
# # for method in methods:
# # print(method)
# for csv in dyndb_csvs[:]:
#     csvpath = os.path.join(stress_ser_path, f'{csv}.csv')
#     df = pd.read_csv(csvpath)

#     instance = '_'.join(csv.split('_')[:-2])


#     sig_pl = dyndb[dyndb['dyn_file_name_original'] == instance]['plateau_stress_g'].values[0]#/10e6
#     W = dyndb[dyndb['dyn_file_name_original'] == instance]['energy_absorbed_g'].values[0]#/10e6

#     pl_end = dyndb[dyndb['dyn_file_name_original'] == instance]['end_strain_g'].values[0]
#     # pl_end_idx = dyndb[dyndb['Strain'] == pl_end].index

#     strn = np.asarray(df['Strain'])
#     pl_end_idx = np.where(np.isclose(pl_end, strn))[0][0]

#     strain = df['Strain'][:pl_end_idx]
#     stress_series = df['stress_bottom_gsreg'][:pl_end_idx]#/10e6


#     result = optimize_loop() #method=method
#     # initial_guess = [A_guess, B_guess, C_guess, m_guess, n_guess]


#     # result = minimize(mod_JC, initial_guess, args=(strain, stress_series, sig_pl, W, pl_end),  method='Powell')

#     A_opt, B_opt, C_opt, m_opt, n_opt = result.x

    
#     offset = -0.01
#     strs_pred = A_opt * (strain + offset)**m_opt + B_opt*((strain + offset)/(C_opt-(strain + offset)))**n_opt

#     W_pred = np.trapz(strs_pred[10:], strn[10:strs_pred.shape[0]])
#     sig_pl_pred = np.mean(strs_pred[200:400])

#     opt_params = [A_opt, B_opt, C_opt, m_opt, n_opt, W_pred, sig_pl_pred]
#     for opt_col, par in zip(opt_cols, opt_params):
#         idx = dyndb[dyndb['dyn_file_name_original'] == instance].index
#         dyndb.loc[idx, opt_col] = par

#     fig = make_subplots(rows=1, cols=1)

#     fig.append_trace(go.Scatter(x = strain, y = stress_series, name='original_data'), row=1, col=1)
#     fig.append_trace(go.Scatter(x = strain, y = strs_pred, name='predicted'), row=1, col=1)
#     fig.update_layout(width=1000)
#     plotpath = os.path.join('/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/fitted_curves_17APR', instance)
#     fig.write_image(f'{plotpath}.png')
    
#     sig_pl_pct_error = np.abs((sig_pl_pred - sig_pl)/sig_pl)*100
#     W_pct_error = np.abs((W_pred - W)/sig_pl)*100

#     print(f"\t{W_pred:.3e}\t{W:.3e}\t\t{W_pct_error:.2f}%\t\t{sig_pl_pred:.3e}\t\t{sig_pl:.3e}\t\t{sig_pl_pct_error:.2f}%\t\t{instance}")
#     print(f"Optimized parameters: {A_opt:.3e}\t{B_opt:.3e}\t{C_opt:.2f}\t{m_opt:.2f}\t{n_opt:.2f}")

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
# dyndb[opt_cols]

In [ ]:
opt_cols

In [ ]:
opt_cols

In [ ]:
# dyndb[(dyndb['W_pct_error']>5) | (dyndb['sig_pl_pct_error']> 5)]

In [ ]:
# dyndb.loc[i][opt_cols].values
dyndb.loc[i, ['W_pct_error', 'sig_pl_pct_error']].values

In [ ]:
for i in dyndb.index:

    move_to_next=False
    
    if dyndb.loc[i, 'W_pct_error'] > 5 or dyndb.loc[i, 'sig_pl_pct_error'] > 5: # !!!!!!!!!! change to "and", and also account for NaNs??... just df.fillna(1e2) ->> no worries about NaNs, ther aren't any
        
        original_error = dyndb.loc[i, ['W_pct_error', 'sig_pl_pct_error']].values

        print(dyndb.loc[i, 'dyn_file_name_original'])
        print(f"Original errors:\tW  {original_error[0]:.2f}%\t\tsig_pl  {original_error[1]:.2f}%")


        # optimize, calculate errors
        csv = dyndb.loc[i, 'dyn_file_name_original'] + '_processed_untrunc'
        csvpath = os.path.join(stress_ser_path, f'{csv}.csv')
        df = pd.read_csv(csvpath)

        instance = '_'.join(csv.split('_')[:-2])


        sig_pl = dyndb[dyndb['dyn_file_name_original'] == instance]['plateau_stress_g'].values[0]#/10e6
        W = dyndb[dyndb['dyn_file_name_original'] == instance]['energy_absorbed_g'].values[0]#/10e6

        pl_end = dyndb[dyndb['dyn_file_name_original'] == instance]['end_strain_g'].values[0]
        # pl_end_idx = dyndb[dyndb['Strain'] == pl_end].index

        strn = np.asarray(df['Strain'])
        pl_end_idx = np.where(np.isclose(pl_end, strn))[0][0]

        strain = df['Strain'][:pl_end_idx]
        stress_series = df['stress_bottom_gsreg'][:pl_end_idx]#/10e6


        for j, temp in enumerate(temps):

            result = optimize_loop(temp = temp) #method=method

            A_opt, B_opt, C_opt, m_opt, n_opt = result.x

            
            offset = -0.01
            strs_pred = A_opt * (strain + offset)**m_opt + B_opt*((strain + offset)/(C_opt-(strain + offset)))**n_opt

            W_pred = np.trapz(strs_pred[10:], strn[10:strs_pred.shape[0]])
            sig_pl_pred = np.mean(strs_pred[200:400])

            sig_pl_pct_error = np.abs((sig_pl_pred - sig_pl)/sig_pl)*100
            W_pct_error = np.abs((W_pred - W)/sig_pl)*100


            if sig_pl_pct_error <= 5.5 and W_pct_error <=5.5:

                move_to_next = True

                opt_params = [A_opt, B_opt, C_opt, m_opt, n_opt, W_pred, sig_pl_pred, W_pct_error, sig_pl_pct_error]


                for opt_col, par in zip(opt_cols, opt_params):
                    # idx = dyndb[dyndb['dyn_file_name_original'] == instance].index
                    dyndb.loc[i, opt_col] = par

                fig = make_subplots(rows=1, cols=1)

                fig.append_trace(go.Scatter(x = strain, y = stress_series, name='original_data'), row=1, col=1)
                fig.append_trace(go.Scatter(x = strain, y = strs_pred, name='predicted'), row=1, col=1)
                fig.update_layout(width=1000)
                plotpath = os.path.join('/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/fitted_curves_17APR', instance)
                fig.write_image(f'{plotpath}.png')
                
                
                print(f"optimal results:\ttemp:\t{temp}\t{instance}")
                print(f"\t{W_pred:.3e}\t{W:.3e}\t\t{W_pct_error:.2f}%\t\t{sig_pl_pred:.3e}\t\t{sig_pl:.3e}\t\t{sig_pl_pct_error:.2f}%")
                print(f"Optimized parameters: {A_opt:.3e}\t{B_opt:.3e}\t{C_opt:.2f}\t{m_opt:.2f}\t{n_opt:.2f}")
                    
                break

            elif j == len(temps)-1:
                print("last iteration reached, no improvement found")
                # # No improvement found, 
                # print(f"results for last iteration:\ttemp:\t{temp}\t{instance}")
                # print(f"\t{W_pred:.3e}\t{W:.3e}\t\t{W_pct_error:.2f}%\t\t{sig_pl_pred:.3e}\t\t{sig_pl:.3e}\t\t{sig_pl_pct_error:.2f}%")
                # print(f"Optimized parameters: {A_opt:.3e}\t{B_opt:.3e}\t{C_opt:.2f}\t{m_opt:.2f}\t{n_opt:.2f}")
                

                # fig = make_subplots(rows=1, cols=1)

                # fig.append_trace(go.Scatter(x = strain, y = stress_series, name='original_data'), row=1, col=1)
                # fig.append_trace(go.Scatter(x = strain, y = strs_pred, name='predicted'), row=1, col=1)
                # fig.update_layout(width=1000)
                # plotpath = os.path.join('/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/fitted_curves_17APR', instance)
                # fig.write_image(f'{plotpath}.png')

                move_to_next = True
        
            if move_to_next:
                break

    # else:
    #     pass

    # if not move_to_next:
    #     pass


In [ ]:
# dbfitpath = os.path.join(nbpath, 'dyn_data', 'fcc_0p35_constit_eqn_params.csv')
dbfitpath = os.path.join(nbpath, 'dyn_data','VF_35_64_VF35p92_constit_eqn_params.csv')

dyndb.to_csv(dbfitpath)

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
from scipy.optimize import dual_

In [ ]:
dyndb[(dyndb['W_pct_error']>5) | (dyndb['sig_pl_pct_error'] >5) | (dyndb['W_pct_error'].isna() )| (dyndb['sig_pl_pct_error'].isna())][['dyn_file_name_original','W_pct_error', 'sig_pl_pct_error']]##.corr()

In [ ]:
fcc_0p10_sr1000
Original errors:	W  6.85%		sig_pl  13.07%
results for last iteration:	temp:	11790	fcc_0p10_sr1000
	2.558e+08	2.837e+08		6.27%		3.908e+08		4.450e+08		12.17%
Optimized parameters: 4.432e+08	1.214e+11	1.90	0.10	12.88

In [ ]:
dyndb[dyndb.dyn_file_name_original == 'fcc_0p10_sr1000'][opt_cols]


In [ ]:
Original errors:	W  8.82%		sig_pl  11.08%
optimal results:	temp:	5500	fcc_0p24_sr1000
	8.944e+08	9.372e+08		2.92%		1.505e+09		1.467e+09		2.63%
Optimized parameters: 1.707e+09	2.937e+11	9.28	0.10	16.19

In [ ]:
# dyndb[dyndb.dyn_file_name_original == 'fcc_0p24_sr1000'][opt_cols]

In [ ]:
dyndb[['dyn_file_name_original',]+opt_cols]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
subdf[['W_pct_error', 'sig_pl_pct_error']].corr()

In [ ]:
fig = make_subplots(rows=1, cols=1)

subdf = dyndb[dyndb['sig_pl_pct_error'] < 5] #W_pct_error

fig.append_trace(go.Scatter(x = subdf.index, y = subdf['W_pct_error'], mode='markers', name='W pct error'), row=1, col=1)
fig.append_trace(go.Scatter(x = subdf.index, y = subdf['sig_pl_pct_error'], mode='markers', name='sig pl pct error'), row=1, col=1)
fig.show()
['W_pct_error', 'sig_pl_pct_error']
# px.scatter(dyndb, x=dyndb.index, y = 'W_pct_error')

In [ ]:
fig = make_subplots(rows=1, cols=1)

subdf = dyndb[dyndb['W_pct_error'] < 50]

fig.append_trace(go.Scatter(x = subdf.index, y = subdf['W_pct_error'], mode='markers', name='W pct error'), row=1, col=1)
fig.append_trace(go.Scatter(x = subdf.index, y = subdf['sig_pl_pct_error'], mode='markers', name='sig pl pct error'), row=1, col=1)
fig.show()
# ['W_pct_error', 'sig_pl_pct_error']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(subdf.W_pct_error)

In [ ]:
subdf.W_pct_error.describe()

In [ ]:
subdf.sig_pl_pct_error.describe()

In [ ]:
# plt.boxplot(subdf.sig_pl_pct_error)

In [ ]:
# plt.hist(subdf.sig_pl_pct_error)

In [ ]:
subdf[['W_pct_error', 'sig_pl_pct_error']].corr()

In [ ]:
csv

In [ ]:
strn

In [ ]:
dyndb